In [1]:
import os
import sys

proj_root = os.path.abspath(os.getcwd())
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

In [4]:
# Zelle 2: HotelManager importieren und Instanz erstellen

from business_logic.hotel_manager import HotelManager
from data_access.hotel_data_access import HotelDataAccess

import model
from model import hotel

# Pfad zur SQLite-Datenbank:
db_path = os.path.join(proj_root, "database", "hotel_reservation_sample.db")

# HotelManager mit dem Datenbankpfad erzeugen
manager = HotelManager(HotelDataAccess(db_path))

1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit
ich dasjenige auswählen kann, welches meinen Wünschen
entspricht. Wünsche sind:


1.1. Ich möchte alle Hotels in einer Stadt durchsuchen,
damit ich das Hotel nach meinem bevorzugten Standort
(Stadt) auswählen kann.


In [9]:
# DAO auf die DB zeigen lassen
dao = HotelDataAccess("database/hotel_reservation_sample.db")
manager = HotelManager(dao)

# nach Stadt filtern im String
hotels_in_WunschStadt = manager.show_hotels_by_city("Lausanne")

if not hotels_in_WunschStadt:
    print("Keine Hotels in dieser Stadt gefunden.")
else:
    for h in hotels_in_WunschStadt:
        print(f"{h.name} – {h.address.street}, {h.address.zip_code}")

Keine Hotels in dieser Stadt gefunden.


1.2. Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

In [36]:
dao = HotelDataAccess("database/hotel_reservation_sample.db")
manager = HotelManager(dao)

# Stadt und min_stars angeben
city = "Basel"
min_stars = 5

# 3) Methode aufrufen
result = manager.show_hotels_by_city_and_min_stars(city, min_stars)

# 4) Ausgabe, falls Treffer vorhanden
if result:
    print(f"Gefundene Hotels in '{city}' mit mindestens {min_stars} Sternen:")
    for h in result:
        print(f"  • {h.name} ({h.stars} Sterne) – {h.address.street}, {h.address.zip_code}")

Gefundene Hotels in 'Basel' mit mindestens 5 Sternen:
  • Les Trois Rois (5 Sterne) – Freiestrasse 10, 4051
